In [42]:
import os
from roboflow import Roboflow
import re
import yaml
import shutil

In [ ]:
RF_KEY  = os.environ.get("RF_KEY", "")
RF_WORKSPACE = os.environ.get("RF_WORKSPACE", "")
RF_PROJECT = os.environ.get("RF_LABELLER_PROJECT", "")

VERSION_NUM = 2
CLASSES = ["concentric_whorl", "imploding_whorl", "loop","standard_arch"]
print(RF_KEY, RF_WORKSPACE, RF_PROJECT)

efcTvzGpArL7OorvmZT8 fingerprint-generation fingerprint-pattern-detection


In [44]:
def filter_images(class_name: str, dir: str):
    labels = {   
        "concentric_whorl": ("0", f"Concentric whorl dataset v{VERSION_NUM}"),
        "imploding_whorl": ("1", f"Imploding whorl dataset v{VERSION_NUM}"),
        "loop": ("2", f"Loop dataset v{VERSION_NUM}"),
        "standard_arch": ("3", f"Simple arch dataset v{VERSION_NUM}")
    }

    label = labels.get(class_name.lower(), None)[0]
    label_dir = os.path.abspath(os.path.join(dir, "labels"))
    image_dir = os.path.abspath(os.path.join(dir, "images"))

    for label_file_name in os.listdir(label_dir):
        label_file_path = os.path.join(label_dir, label_file_name)

        with open(label_file_path) as label_file:
            lines = label_file.read(1)
            try:
                if lines[0] != label:
                    os.remove(label_file_path)
                    image_file_name = re.sub(r"\.[^.]+$", "", label_file_name)+".jpg"
                    image_file_path = os.path.join(image_dir, image_file_name)
                    os.remove(image_file_path)
            except IndexError: # catches any images labelled as "null" on Roboflow
                    os.remove(label_file_path)
                    image_file_name = re.sub(r"\.[^.]+$", "", label_file_name)+".jpg"
                    image_file_path = os.path.join(image_dir, image_file_name)
                    os.remove(image_file_path)


In [45]:
def modify_yaml(class_name, dir):
    yaml_dir = os.path.abspath(os.path.join(dir, "data.yaml"))

    with open(yaml_dir, "r") as f:
        data = yaml.safe_load(f)
    data['names'] = [class_name]
    data['nc'] = 1
    with open(yaml_dir, 'w') as f:
        yaml.dump(data, f, sort_keys=False)    

In [46]:
def filter_for_class(dir:str, class_name:str):
    filter_images(class_name, os.path.abspath(os.path.join(dir, "train/")))
    filter_images(class_name, os.path.abspath(os.path.join(dir, "valid/")))
    filter_images(class_name, os.path.abspath(os.path.join(dir, "test/")))
    modify_yaml(class_name, dir)
    new_dir = os.path.abspath(os.path.join(dir, f"../v{VERSION_NUM}/{class_name} dataset v{VERSION_NUM}"))
    os.rename(dir, new_dir)

In [ ]:
def filter_class_dataset(class_name: str):
    rf = Roboflow(api_key=RF_KEY)
    project = rf.workspace(RF_WORKSPACE).project(RF_PROJECT)
    dataset = project.version(VERSION_NUM).download("yolov8", f"../../datasets/Fingerprint pattern/Fingerprint dataset v{VERSION_NUM}")

    curr_dir = os.getcwd()
    parent_dir = os.path.abspath(os.path.join(curr_dir, f"../.."))
    dataset_dir = os.path.abspath(os.path.join(parent_dir, f"datasets/Fingerprint pattern/Fingerprint dataset v{VERSION_NUM}" ))
    filter_for_class(dir = dataset_dir, class_name=class_name)

In [52]:
curr_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(curr_dir, f"../.."))
new_dir = os.path.abspath(os.path.join(parent_dir, f'datasets/Fingerprint pattern/v{VERSION_NUM}'))
print(new_dir)
if os.path.exists(new_dir):
    os.remove(new_dir)
os.makedirs(new_dir)

for c in CLASSES:
    try:
        filter_class_dataset(c)
    except:
        pass


/Users/jin/Documents/GitHub/datasets/Fingerprint pattern/v2
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v2 in yolov8:: 100%|██████████| 436/436 [00:00<00:00, 8067.21it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v2 in yolov8:: 100%|██████████| 436/436 [00:00<00:00, 7901.85it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v2 in yolov8:: 100%|██████████| 436/436 [00:00<00:00, 8165.26it/s]
